In [ ]:
-- TODO set up dummy network
-- TODO set up mask-making code
-- TODO set up criterion code 
-- TODO test noopmask+criterion against regular critierion
-- TODO test real mask with dummy annealing
-- TODO test real thing


In [ ]:
input = torch.rand(1,32,32) -- pass a random tensor as input to the network
output = net:forward(input)
--print(input)
print(output:exp())

In [ ]:
criterion = nn.ClassNLLCriterion() -- a negative log-likelihood criterion for multi-class classification
error = criterion:forward(output, 3) -- let's say the groundtruth was class number: 3
gradients = criterion:backward(output, 4)
gradInput = net:backward(input, gradients)
print(error)
print(criterion)
print(gradients)

In [ ]:
asdf = torch.ones(10)
blah = torch.zeros(10)
blah[7] = 1
qwer = torch.cmul(blah, asdf)
print(blah)
print(asdf)
print(qwer)

In [2]:
-- TODO I don't think using cmul is correct. You have to make sure you multiply each result forward by the mask.
-- TODO test this out on some REAL data from provider.

-- TODO there are special functions for masks, e.g. maskedCopy, maskedFill, try those...

require 'nn';

do

-- Construct a new one of these every mini batch, because you need a new unlabledMask for each one.
-- This assumes unlabeled have labeled of 0.
local PseudoLabelCriterion, Criterion = torch.class('nn.PseudoLabelCriterion', 'nn.Criterion')

-- crit: Criterion object
-- unlabeledMask: a 1 x n tensor, where n is the mini-batch size.
--                It has zeros for labeled data, and ones for unlabeled data.
-- kAnneal: a scalar we use to weight the unlabled components of the gradient of the loss 
function PseudoLabelCriterion:__init(crit, unlabeledMask, kAnneal)
   Criterion.__init(self)
   self.crit = crit
   self.unlabeledMask = unlabeledMask -- TODO maybe this should be a long()
   -- The unlabled mask, but the ones are replaced with kAnneal, and zeros are replaced with ones.
        -- TODO do I have to cuda this?
   if (type(unlabeledMask) == 'number') then
      self.annealedUnlabeledMask = self.unlabeledMask*(kAnneal-1) + 1
   else
      -- putting this into a diagonal matrix allows you to multiply rows easily
      self.annealedUnlabeledMask = self.unlabeledMask*(kAnneal-1) + torch.ones(self.unlabeledMask:size())
   end
end


-- returns/sets the error
-- NOTE: modifies target. we assume target has zeros for the unlabeled data.
-- input must be n x k matrix, where n is the mini batch size and k is the number of classes.
-- target must be a 1 x n matrix with the correct labels
function PseudoLabelCriterion:updateOutput(input, target)
   -- squeeze gets rid of extra dimensions.
   input = input:squeeze()
   if type(target) == 'number' then -- single dimension case
      if unlabeledMask == 1 then
         _, predictedLabel = torch.max(input, 1)
         target = predictedLabel
      end
      self.crit:updateOutput(input, target)
      self.crit.output = self.crit.output * self.annealedUnlabeledMask
   else
      target = target:squeeze()
      -- Get the max indexes of the target, which gets you the predicted labels      
      _, predictedLabels = torch.max(input, 2)
      predictedLabels = predictedLabels:float() -- TODO do I have to cuda this?
      -- keep the predicted labels of the unlabeled data.
      -- This assumes the unlabeled data has a target of zero.   
      target:add(torch.cmul(predictedLabels, self.unlabeledMask))
      self.crit:updateOutput(input, target)
      -- multiplies unlabeled entries by kAnneal.      
      self.crit.output:cmul(self.annealedUnlabeledMask)
   end
   self.output = self.crit.output
   return self.output
end

-- TODO no idea if this is correct.
-- returns/sets the gradient
function PseudoLabelCriterion:updateGradInput(input, target)
   input = input:squeeze()
   if type(target) == 'number' then 
      self.crit:updateGradInput(input, target)
      self.crit.gradInput = self.crit.gradInput * self.annealedUnlabeledMask      
   else
      target = target:squeeze()
      self.crit:updateGradInput(input, target)
      -- TODO SHOULD THIS BE MUL???        
            -- TODO do you nead to take the diag + transpose here?
      self.crit.gradInput:cmul(self.annealedUnlabeledMask) -- multiplies unlabeled entries by kAnneal.
   end     
   self.gradInput = self.crit.gradInput
   return self.gradInput
end

end



In [3]:
require 'provider'

p = load_provider('tiny', 'evaluate', false)
data = p.trainData.data
labels = p.trainData.labels

==> loading data	
provider.tiny.evaluate.t7	
loading data from file...	


In [4]:
model = nn.Sequential()
model:add(dofile('models/sample.lua'))
model:training()

In [38]:
batchSize = 3
batch = torch.range(1, batchSize):long()
inputs = data:index(1, batch)
targets = torch.FloatTensor(batchSize)
-- targets are all ones to start
targets:copy(labels:index(1, batch))
targets[2] = 0 -- make the 2nd one unlabeled

predictions = model:forward(inputs)

print(predictions)
print(targets)

-0.0070  0.0114 -0.5080 -0.2450 -0.0026  0.4792  0.0237  0.3736 -0.4440  0.3248
-0.0490 -0.0236  0.1370 -0.0756 -0.6475  0.2812  0.6069 -0.2230  0.1334  0.1203
 0.0495  0.2698 -0.0844  0.1220 -0.3436  0.0022  0.0825 -0.6721  0.0897 -0.2334
[torch.FloatTensor of size 3x10]

 1
 0
 1
[torch.FloatTensor of size 3]



In [ ]:
-- TODO 

unlabeledMask = torch.zeros(3)
unlabeledMask[2] = 1 -- 2nd one unlabeled
print('unlabeled mask')
print(unlabeledMask)
kAnneal = 2

_, predictedLabels = torch.max(predictions, 2)
predictedLabels = predictedLabels:float()
print('model predictions')
print(predictedLabels)

print('original targets')
print(targets)

pseudoTargets = torch.C

In [55]:
-- TODO 

unlabeledMask = torch.zeros(3)
unlabeledMask[2] = 1 -- 2nd one unlabeled
print('unlabeled mask')
print(unlabeledMask)
kAnneal = 2

_, predictedLabels = torch.max(predictions, 2)
predictedLabels = predictedLabels:float()
print('model predictions')
print(predictedLabels)

print('original targets')
print(targets)

pseudoTargets = torch.FloatTensor(targets:size())
pseudoTargets:copy(targets)
pseudoTargets:add(torch.cmul(predictedLabels, unlabeledMask))

print('pseudo targets')
print(pseudoTargets)

criterion = nn.CrossEntropyCriterion()
error = criterion:forward(predictedLabels, pseudoTargets)
print('error')
print(error)

print('annealed error')
annealedUnlabeledMask = unlabeledMask*(kAnneal-1) + torch.ones(unlabeledMask:size())
error:cmul(annealedUnlabeledMask)
print(error)

-- TODO OH SHIT. the error here isn't a vector with errors for each data point.
-- Error is computed over all the data points in the batch
-- fuuuuuuuuuuuuck this kills the whole criterion idea

unlabeled mask	
 0
 1
 0
[torch.FloatTensor of size 3]

model predictions	
 6
 7
 2
[torch.FloatTensor of size 3x1]

original targets	
 1
 0
 1
[torch.FloatTensor of size 3]



[string "-- TODO ..."]:17: attempt to call field 'CudaTensor' (a nil value)
stack traceback:
	[string "-- TODO ..."]:17: in main chunk
	[C]: in function 'xpcall'
	/Users/pinesol/torch/install/share/lua/5.1/itorch/main.lua:179: in function </Users/pinesol/torch/install/share/lua/5.1/itorch/main.lua:143>
	/Users/pinesol/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...s/pinesol/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...s/pinesol/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...s/pinesol/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/pinesol/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0108ab1d50: 

In [ ]:
-- TODO test jacobian

-- parameters
local precision = 1e-5
local jac = nn.Jacobian

-- define inputs and module
local ini = math.random(10,20)
local inj = math.random(10,20)
local ink = math.random(10,20)
local percentage = 0.5
local input = torch.Tensor(ini,inj,ink):zero()
local module = nn.Dropout(percentage)

-- test backprop, with Jacobian
local err = jac.testJacobian(module,input)
print('==> error: ' .. err)
if err<precision then
   print('==> module OK')
else
      print('==> error too large, incorrect implementation')
end

